In [30]:
import csv
from itertools import islice, combinations
from collections import defaultdict
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
dataset_counts = defaultdict(int)
img_counts = defaultdict(lambda : defaultdict(int))
unique_imgs = defaultdict(set)
img_votes = defaultdict(lambda: defaultdict(lambda: [0,0])) #no,yes tuples by dataset and image

with open('../all_gans_inf.csv') as f:
    reader = csv.DictReader(f)
    for row in islice(reader, None):
        img = row['img']        
#         dataset_name = img[:img.index('/')]
        split_name = img.split('/')
        assert(len(split_name) == 2)
        dataset_name, img_name = split_name
        
        vote_index = 1 if row['correctness'] == row['realness'] else 0
        img_votes[dataset_name][img_name][vote_index] += 1
        
        dataset_counts[dataset_name] += 1
        img_counts[dataset_name][img_name] += 1
unique_imgs = {dataset: len(img_counts[dataset]) for dataset in img_counts}

In [3]:
trues = 0
falses = 0
for f,t in img_votes['progan5000'].values():
    for rating_pair in combinations(f * [0] + t * [1], 2):
        if rating_pair[0] == rating_pair[1]: trues += 1
        else: falses += 1
print(trues, falses)

588 310


In [4]:
trues/(falses + trues)

0.6547884187082406

In [5]:
trues = 0
falses = 0
for f,t in img_votes['styleganceleba5000'].values():
    for rating_pair in combinations(f * [0] + t * [1], 2):
        if rating_pair[0] == rating_pair[1]: trues += 1
        else: falses += 1
print(trues, falses, trues/(falses+trues))

1330 928 0.5890168290522586


In [11]:
nvidia_score = {}
with open('nvidia_realism_scores.csv') as f:
    reader = csv.reader(f)
    for row in islice(reader, 1, None):
        nvidia_score[row[0]] = float(row[1])

In [37]:
nvidia_progan = [(k,v) for k,v in nvidia_score.items() if 'proGAN' in k]
nvidia_stylegan = [(k,v) for k,v in nvidia_score.items() if 'styleGAN' in k]
nvidia_began = [(k,v) for k,v in nvidia_score.items() if 'BEGAN' in k]
nvidia_wgan = [(k,v) for k,v in nvidia_score.items() if 'wgan' in k]

len(nvidia_progan), len(nvidia_stylegan), len(nvidia_began), len(nvidia_wgan), len(nvidia_score)

(223, 310, 196, 425, 1154)

In [71]:
print(\
np.mean([x[1] > 1 for x in nvidia_progan]),
np.mean([x[1] > 1 for x in nvidia_stylegan]),
np.mean([x[1] > 1 for x in nvidia_began]),
np.mean([x[1] > 1 for x in nvidia_wgan])
)

0.34977578475336324 0.5870967741935483 0.23469387755102042 0.1388235294117647


In [45]:
gan_models = ['progan', 'stylegan', 'began', 'wgan']

dense_labels = {}
#dense_labels['val'] = {}
#dense_labels['test'] = {}

for m in gan_models:
    for val_or_test in ('val', 'test'):
        with open('dense_labels/{}_{}_dense_labels.csv'.format(m, val_or_test)) as f:
            reader = csv.reader(f)
            for row in islice(reader,None):
                dense_labels[row[0]] = round(float(row[1]))
                #print(row[0], float(row[1]))

len(dense_labels)

1592

In [54]:
sets = ['progan_val', 'stylegan_val', 'began_val', 'wgan_val',
        'progan_test','stylegan_test','began_test','wgan_test']

file_strings = ['proGAN', 'styleGAN', 'BEGAN', 'wgan',
                'proGAN', 'styleGAN', 'BEGAN', 'wgan',]

set_files = ['dense_labels/cleaned_human_hype_train_scores.csv',
         './stylegan_human_scores_cleaned.csv',
        './dense_labels/run_2_processed_human_scores.csv',
        './dense_labels/run_3_processed_human_scores.csv',
        './dense_labels/run_4_processed_human_scores.csv',
        './dense_labels/run_5_processed_human_scores.csv',
        './dense_labels/run_6_processed_human_scores.csv',
        './dense_labels/run_7_processed_human_scores.csv',
        ]

In [56]:
img_yes_votes = defaultdict(int)
img_all_votes = defaultdict(int)

for i in range(8):
    set_name = sets[i]
    file_string = file_strings[i]
    set_file = set_files[i]
    
    with open(set_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in islice(reader,1,None):
            filename = row[0][5:]

            if file_string not in filename: continue
            assert(row[0][:5] == 'fake_')

            human_label = row[1]
            assert(human_label in ('real', 'fake'))

            img_all_votes[filename] += 1
            if human_label == 'real':
                img_yes_votes[filename] += 1


In [51]:
img

'/Users/alexkolchinski/git/humanception-score/distances'

In [62]:
def img_model(filename):
    if 'proGAN' in filename: return 'progan'
    if 'styleGAN' in filename: return 'stylegan'
    if 'BEGAN' in filename: return 'began'
    if 'wgan' in filename: return 'wgan'

In [63]:
agrees = defaultdict(int)
disagrees = defaultdict(int)

for img, nv in nvidia_score.items():
    nvidia_guess = nv > 1.
    yes_votes = img_yes_votes[img]
    no_votes = img_all_votes[img] - yes_votes
    model = img_model(img)
    
    for i in range(yes_votes):
        if nvidia_guess: agrees[model] += 1
        else: disagrees[model] += 1
    
    for i in range(no_votes):
        if nvidia_guess: disagrees[model] += 1
        else: agrees[model] += 1


In [64]:
agrees

defaultdict(int, {'began': 733, 'wgan': 835, 'progan': 570, 'stylegan': 526})

In [65]:
disagrees

defaultdict(int, {'began': 247, 'wgan': 165, 'progan': 430, 'stylegan': 474})

In [67]:
agreement_rates = {k:v/(v+disagrees[k]) for k,v in agrees.items()}
agreement_rates

{'began': 0.7479591836734694, 'wgan': 0.835, 'progan': 0.57, 'stylegan': 0.526}